In [5]:
# 하나만 테스트 해본 것. 문제 생길 시에 참고하자

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import csv
import re

year = 0
data = []
stocks = []
link = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=161890'

# 브라우저 생성
browser = webdriver.Chrome()
time.sleep(3)
browser.get('{}'.format(link))


elementXpath = '//*[@id="pArea"]/div[1]/div/table/tbody/tr[1]/td/dl/dt[1]/span'
title = browser.find_element(by=By.XPATH, value = elementXpath).text

# 포괄손익계산서 이동하기
browser.find_element_by_link_text("재무분석").click()
browser.find_element_by_link_text("포괄손익계산서").click()

time.sleep(1)
# 영업이익 비교하기_ 조건 불합시 continue
elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[{}]'.format(3+year)
O_profitAfter = browser.find_element(by=By.XPATH, value = elementXpath).text

elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[{}]'.format(2+year)
O_profitBefore = browser.find_element(by=By.XPATH, value = elementXpath).text
print("영업 이익", O_profitAfter, O_profitBefore)

# 재무상태표 이동하기
browser.find_element_by_link_text("재무상태표").click()
time.sleep(1)

# 순운전자본 구하기
elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[{}]'.format(3+year)
current_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[128]/td[{}]'.format(3+year)
current_liabilities = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

net_working_capital = current_asset - current_liabilities
print('순운전자본', round(net_working_capital))

# 조건으로부터 살아남았다면, 리스트 테이블에 추가
stocks.append(title)
data.append(net_working_capital)
browser.quit()

# 시가총액 구하기
# 브라우저 생성 
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome()

# 반복문 시작
data2 = []
stocks2 = []
for i in range(len(stocks)):
    time.sleep(3)
    browser.get('{}'.format('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101'))

    # 원하는 곳으로 이동
    elementXpath = '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[2]/ul/li[1]/ul/li[3]/a'
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    browser.find_element(by = By.XPATH , value = elementXpath).click()

    # 검색어 입력
    elementXpath = "//*[@id='tboxisuCd_finder_stkisu0_1']"
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    search_box = browser.find_element(by = By.XPATH , value = elementXpath)
    browser.find_element(by = By.XPATH , value = elementXpath).clear()
    search_box.send_keys("{}".format(stocks[i]))
    search_box.send_keys(Keys.RETURN)

    # 새로운 창에서의 클릭
    browser.switch_to.window(browser.window_handles[-1])

    elementXpath = "//*[@id='jsGrid__finder_stkisu0_1']/tbody/tr[1]/td[2]"
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)
    browser.find_element(by = By.XPATH , value = elementXpath).click()

    # 다시 원래 창으로 돌아옴.
    browser.switch_to.window(browser.window_handles[0])

    # 날짜 선택 먼저 20181231 + year
    box = browser.find_element(by = By.XPATH , value = " //*[@id='endDd']")
    box.clear()
    box.send_keys("20{}1231".format(18+year))

    # 1일 버튼 클릭
    browser.find_element(by = By.XPATH , value = "//*[@id='MDCSTAT017_FORM']/div[1]/div/table/tbody/tr[2]/td/div/div/button[2]").click()

    # '조회' 버튼 클릭
    browser.find_element(by = By.XPATH , value = '//*[@id="jsMdiTab"]/li[1]/a/button').click()
    browser.find_element(by = By.XPATH , value = "//*[@id='jsSearchButton']").click()

    # 출력 
    elementXpath = "//*[@id='jsMdiContent']/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr/td[10]"
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    cap = int(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(",","")) // (10**8)
    rate = round(cap/data[i]*100,2)
    print('시가총액:',int(cap))
    print('시가총액/순운전자본', str(rate)+'%')
    data2.append([stocks[i], str(rate)+'%'])
    stocks2.append(stocks[i])

browser.quit()
    
# 브라우저 생성
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome()

for i in range(len(stocks2)):
    time.sleep(3)
    browser.get('{}'.format('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101'))

    # 원하는 곳으로 이동
    elementXpath = '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[2]/ul/li[1]/ul/li[4]/a'
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    browser.find_element(by = By.XPATH , value = elementXpath).click()

    # 옵션 선택
    elementXpath = '//*[@id="MDCSTAT018_FORM"]/div[1]/div/table/tbody/tr[1]/td/label[2]'
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    browser.find_element(by = By.XPATH , value = elementXpath).click()

    # 검색어 입력
    elementXpath = "//*[@id='tboxisuCd_finder_stkisu0_1']"    
    search_box = browser.find_element(by = By.XPATH , value = elementXpath)
    search_box.clear()
    search_box.send_keys("{}".format(stocks2[i]))
    search_box.send_keys(Keys.RETURN)

    # 새로운 창에서의 클릭
    browser.switch_to.window(browser.window_handles[-1])

    elementXpath = "//*[@id='jsGrid__finder_stkisu0_1']/tbody/tr[1]/td[2]"
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
    time.sleep(1)

    browser.find_element(by = By.XPATH , value = elementXpath).click()

    # 다시 원래 창으로 돌아옴.
    browser.switch_to.window(browser.window_handles[0])

    # 드롭다운 날짜 선택
    select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="strtYear"]'))
    select_fr.select_by_index(4-year)
    select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="strtMonth"]'))
    select_fr.select_by_index(11)

    select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="endYear"]'))
    select_fr.select_by_index(3-year)
    select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="endMonth"]'))
    select_fr.select_by_index(11)

    # '조회' 버튼 클릭
    browser.find_element(by = By.XPATH , value = '//*[@id="jsMdiTab"]/li[1]/a/button').click()
    browser.find_element(by = By.XPATH , value = "//*[@id='jsSearchButton']").click()

    # 수익률
    try:
        elementXpath = '//*[@id="jsMdiContent"]/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr[1]/td[4]'
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
        time.sleep(1)

        stock_before = float(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(',',''))
    except:
        continue

    try:
        elementXpath = '//*[@id="jsMdiContent"]/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr[13]/td[4]'
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
        time.sleep(1)

        stock_after = float(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(',',''))
    except:
        continue

    earning_rate = str(round((stock_after-stock_before)/stock_before*100 , 2)) + "%"
    print('수익률 :', earning_rate)
    data2[i].append(int(stock_before))
    data2[i].append(int(stock_after))
    data2[i].append(earning_rate)
    print(data2[i])


0.003
